In [ ]:
import requests as r
import pandas as pd
import sys
sys.path.append('../')
# print(sys.path)
import defillama_utils as dfl
sys.path.pop()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [ ]:
# wee = dfl.get_protocol_tvls()
# display(wee)

protocols = r.get('https://api.llama.fi/lite/protocols2', headers={'Content-Type': 'application/json'}).json()
print(protocols)

In [ ]:
all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
res = res[res['slug'].isin(['pooltogether'])]
res = res.T
# display(res)

In [ ]:
protocols = r.get('https://api.llama.fi/lite/protocols2', headers=header).json()['protocols']
# doublecounted_values = [protocol["chainTvls"]["doublecounted"] for protocol in protocols]
# print(protocols)
# for protocol in protocols:
#         print(protocol['chainTvls'].keys())
doublecounted_protocols = [protocol for protocol in protocols if any("-doublecounted" in key for key in protocol["chainTvls"])]
doublecounted_protocol_names = [element["name"] for element in doublecounted_protocols]
print(doublecounted_protocol_names)

In [ ]:
def get_protocol_tvls(min_tvl = 0, excluded_cats = ['CEX','Chain']):#,excluded_flags = ['staking','pool2']):
        all_api = 'https://api.llama.fi/protocols'
        resp = pd.DataFrame( r.get(all_api, headers=header).json() )
        resp = resp[resp['tvl'] > min_tvl ] ##greater than X
        if excluded_cats != []: #If we have cagtegories to exclude
                resp = resp[~resp['category'].isin(excluded_cats)]
        # Get Other Flags
        doublecounts = get_doublecounted_protocol_names()
        resp = resp.assign(is_doubelcount = resp['name'].isin(doublecounts))
        # if excluded_flags != []: #If we have cagtegories to exclude
        #         for flg in excluded_flags:
        #                 resp = resp[resp[flg] != True]
        return resp